The requirements came in, its a quiz app. Nothing like you've seen before.

### Objectives
```
User should be able to
- Start the quiz
- Choose an option to answer

On the result page display the total score
```

You being a smart developer(by making a mess in the past with orphan states across the app) start with a state manager.

What do you start with?
- Actions

What are `Actions`?
- User interactable actions
- Events in the application logic

What are the actions required in the quiz app?
- User interactions
    - Start quiz
    - Choose answer
- Events in the application
    - End quiz

    
## Actions
| Action Name | Action Description |
|-------------|--------------------|
| `START_QUIZ`  | User starts the quiz |
| `CHOOSE_ANSWER` | User clicks an option out of the given options |
| `END_QUIZ` | Triggered when there are no more questions |

What does an action look like?
- Its a javascript object
```json
{
    "type": String,
    "payload": JSONValue
}
```
- The `type` is a string constant typically representing an action name
- The `payload` is the data associated with the action event
Eg
```json
{
    "type": "ADD_POST",
    "payload": {
        "tags": ["solution"]
        "text": "You are a developer..."
    }
}
```

Whats the next step?
- `Reducers`

What are reducers?
- Reducers manage your state using `prevstate` and `action` to give you `nextstate`

`PrevState + Action => NextState`

What do they look like?
- Here's a trivial example of a reducer
```javascript
const initialState = 0
function counterReducer(state = initialState, action){
    switch(action.type){
        case "INCREMENT":
            return state + 1
        case "DECREMENT":
            return state - 1
        default:
            return state
    }
}
```

How to decide what reducers to make?
- Every reducer in the store(global state) is associated with a key

Eg - The value returned by `counterReducer` will be associated with key `count` in global state.
```json
{
    "count": counterReducer
}
```

- So one good way to figure out the reducers is by determining the states you will access.

What are the states we need for quiz app?
- We need a way to track 
    - the questions
    - score

    
## Reducers
| Reducer Name | Reducer Function |
|-|-|
| questionReducer  | To track the current question |
| scoreReducer | To track count of correctly answered questions |

Quiz data.

https://github.com/googlearchive/android-Quiz/blob/master/Application/src/main/assets/Quiz.json

*Credit - googlearchive/android-Quiz*

In [1]:
const initialQuestionsState = [
    {
        "question": "What is the scientific name of a butterfly?",
        "answers": [
            "Apis",
            "Coleoptera",
            "Formicidae",
            "Rhopalocera"
        ],
        "correctIndex": 3
    },
    {
        "question": "How hot is the surface of the sun?",
        "answers": [
            "1,233 K",
            "5,778 K",
            "12,130 K",
            "101,300 K"
        ],
        "correctIndex": 1
    },
    {
        "question": "Who are the actors in The Internship?",
        "answers": [
            "Ben Stiller, Jonah Hill",
            "Courteney Cox, Matt LeBlanc",
            "Kaley Cuoco, Jim Parsons",
            "Vince Vaughn, Owen Wilson"
        ],
        "correctIndex": 3
    },
    {
        "question": "What is the capital of Spain?",
        "answers": [
            "Berlin",
            "Buenos Aires",
            "Madrid",
            "San Juan"
        ],
        "correctIndex": 2
    },
    {
        "question": "What are the school colors of the University of Texas at Austin?",
        "answers": [
            "Black, Red",
            "Blue, Orange",
            "White, Burnt Orange",
            "White, Old gold, Gold"
        ],
        "correctIndex": 2
    }
]

The `questionReducer` will track the current question.
The current question will be the first element in `question` state.

`questionReducer([a, b, ...], {type: "CHOOSE_ANSWER"})` => `[b, ...]`

In [2]:
function questionReducer(state = initialQuestionsState, action){
    switch(action.type){
        case "CHOOSE_ANSWER":
            return state.slice(1)
        default:
            return state
    }
}

Testing `questionReducer`

In [3]:
questionReducer(initialQuestionsState, {type: "CHOOSE_ANSWER"})

[ { question: 'How hot is the surface of the sun?',
    answers: [ '1,233 K', '5,778 K', '12,130 K', '101,300 K' ],
    correctIndex: 1 },
  { question: 'Who are the actors in The Internship?',
    answers:
     [ 'Ben Stiller, Jonah Hill',
       'Courteney Cox, Matt LeBlanc',
       'Kaley Cuoco, Jim Parsons',
       'Vince Vaughn, Owen Wilson' ],
    correctIndex: 3 },
  { question: 'What is the capital of Spain?',
    answers: [ 'Berlin', 'Buenos Aires', 'Madrid', 'San Juan' ],
    correctIndex: 2 },
  { question:
     'What are the school colors of the University of Texas at Austin?',
    answers:
     [ 'Black, Red',
       'Blue, Orange',
       'White, Burnt Orange',
       'White, Old gold, Gold' ],
    correctIndex: 2 } ]

`scoreReducer` will track the count of correct answered questions.
`payload.answerStatus` will contain answer state. `true` for `Correct` and `false` for `Incorrect` 

`scoreReducer(x, {type: "CHOOSE_ANSWER", payload: {answerStatus: true}})` => `x + 1`

`scoreReducer(x, {type: "CHOOSE_ANSWER", payload: {answerStatus: false}})` => `x`

In [4]:
const initialScore = 0

In [5]:
function scoreReducer(state = initialScore, action){
    switch(action.type){
        case "CHOOSE_ANSWER":
            if (action.payload.answerStatus == true)
                return state + 1
            return state
        default:
            return state
    }
}

Testing `scoreReducer`

In [6]:
scoreReducer(0, {type: "CHOOSE_ANSWER", payload: {answerStatus: true}})

0

In [7]:
scoreReducer(0, {type: "CHOOSE_ANSWER", payload: {answerStatus: false}})

0

Would it be convenient if we had helper functions to create the action objects?
- Yes, they are called action creators

How about the action names?
- We can define them as constants 

## Action name constants

In [8]:
const START_QUIZ =  "START_QUIZ"
const CHOOSE_ANSWER =  "CHOOSE_ANSWER"
const END_QUIZ =  "END_QUIZ"

## Action creator 

In [9]:
function chooseAnswer(choice, answer){
    if (choice == answer)
        return {
            type: "CHOOSE_ANSWER",
            payload: {
                answerStatus: true
            }
        }
    return {
        type: "CHOOSE_ANSWER",
        payload: {
            answerStatus: false
        }
    }
}

In [10]:
const redux = require('redux')

Do we need combineReducers?
- Yes, since we have multiple reducers, and a store can only have 1 reducer

In [11]:
const { combineReducers } = redux

Every reducer is associated with a key in global state

In [12]:
let rootReducer = combineReducers({
    questions: questionReducer,
    score: scoreReducer
})

Whats the next part?
- Store

Whats a store?
- The store is the global state

In [13]:
const { createStore } = redux

In [14]:
let store = createStore(rootReducer)

In [15]:
store.getState()

{ questions:
   [ { question: 'What is the scientific name of a butterfly?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'How hot is the surface of the sun?',
       answers: [Array],
       correctIndex: 1 },
     { question: 'Who are the actors in The Internship?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'What is the capital of Spain?',
       answers: [Array],
       correctIndex: 2 },
     { question:
        'What are the school colors of the University of Texas at Austin?',
       answers: [Array],
       correctIndex: 2 } ],
  score: 0 }

In [16]:
store.getState().questions[0]

{ question: 'What is the scientific name of a butterfly?',
  answers: [ 'Apis', 'Coleoptera', 'Formicidae', 'Rhopalocera' ],
  correctIndex: 3 }

In [17]:
store.dispatch(chooseAnswer(2, 3))

{ type: 'CHOOSE_ANSWER', payload: { answerStatus: false } }

In [18]:
store.getState()

{ questions:
   [ { question: 'How hot is the surface of the sun?',
       answers: [Array],
       correctIndex: 1 },
     { question: 'Who are the actors in The Internship?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'What is the capital of Spain?',
       answers: [Array],
       correctIndex: 2 },
     { question:
        'What are the school colors of the University of Texas at Austin?',
       answers: [Array],
       correctIndex: 2 } ],
  score: 0 }

So far so good.

What happens when we reach end of questions?
- The quiz ends

Do we have any indicator for ending?
- No

Do we need a state indicating quiz status `START` or `END`?
- Yes, lets call it `status`. `true` will indicate running and `false` will indicate end

In [19]:
const initialQuizStatus = false

In [20]:
function quizStatusReducer(state = initialQuizStatus, action){
    switch(action.type){
        case "START_QUIZ":
            return true
        case "END_QUIZ":
            return false
        default:
            return state
    }
}

In [21]:
rootReducer = combineReducers({
    questions: questionReducer,
    score: scoreReducer,
    status: quizStatusReducer
})

[Function: combination]

In [22]:
store = createStore(rootReducer)

{ dispatch: [Function: dispatch],
  subscribe: [Function: subscribe],
  getState: [Function: getState],
  replaceReducer: [Function: replaceReducer],
  '@@observable': [Function: observable] }

In [23]:
store.getState()

{ questions:
   [ { question: 'What is the scientific name of a butterfly?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'How hot is the surface of the sun?',
       answers: [Array],
       correctIndex: 1 },
     { question: 'Who are the actors in The Internship?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'What is the capital of Spain?',
       answers: [Array],
       correctIndex: 2 },
     { question:
        'What are the school colors of the University of Texas at Austin?',
       answers: [Array],
       correctIndex: 2 } ],
  score: 0,
  status: false }

In [24]:
store.dispatch({type: "START_QUIZ"})

{ type: 'START_QUIZ' }

In [26]:
store.getState()

{ questions:
   [ { question: 'What is the scientific name of a butterfly?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'How hot is the surface of the sun?',
       answers: [Array],
       correctIndex: 1 },
     { question: 'Who are the actors in The Internship?',
       answers: [Array],
       correctIndex: 3 },
     { question: 'What is the capital of Spain?',
       answers: [Array],
       correctIndex: 2 },
     { question:
        'What are the school colors of the University of Texas at Austin?',
       answers: [Array],
       correctIndex: 2 } ],
  score: 0,
  status: true }